In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error, accuracy_score, precision_score,recall_score, f1_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [36]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [38]:
jd = pd.read_csv('jd.csv')
jd.head()

,Unnamed: 0,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [39]:
jd = jd.drop(columns=['Transaction unique identifier', 'ADDRESS','Unnamed: 0'])
jd.shape

(115372, 51)

In [40]:
#dopped column related future improvement potential
jd = jd.drop(columns=['ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_POTENTIAL','POTENTIAL_ENERGY_RATING',
                     'POTENTIAL_ENERGY_EFFICIENCY', 'CO2_EMISSIONS_POTENTIAL','LIGHTING_COST_POTENTIAL','HEATING_COST_POTENTIAL',
                     'HOT_WATER_COST_POTENTIAL'])
jd.shape

(115372, 43)

In [41]:
#Creating list for setting the priority in ordinal encodign
HOT_WATER_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']

In [42]:
CURRENT_ENERGY_RATING = ['G','F','E','D','C','B','A']

In [43]:
HOT_WATER_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']

In [44]:
WINDOWS_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
WINDOWS_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
WALLS_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
WALLS_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
MAINHEAT_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
MAINHEAT_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
MAINHEATC_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
MAINHEATC_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
LIGHTING_ENERGY_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']
LIGHTING_ENV_EFF = ['N/A', 'Very Poor', 'Poor', 'Average', 'Good', 'Very Good']


In [45]:
#Ordinal Encoding
od = OrdinalEncoder(categories = [CURRENT_ENERGY_RATING, HOT_WATER_ENERGY_EFF,HOT_WATER_ENV_EFF, MAINHEAT_ENERGY_EFF,
                                  MAINHEAT_ENV_EFF, LIGHTING_ENERGY_EFF ,LIGHTING_ENV_EFF,MAINHEATC_ENERGY_EFF ,MAINHEATC_ENV_EFF,
                                  WINDOWS_ENERGY_EFF,WINDOWS_ENV_EFF,WALLS_ENERGY_EFF, WALLS_ENV_EFF ])

In [46]:
jd[['CURRENT_ENERGY_RATING', 'HOT_WATER_ENERGY_EFF','HOT_WATER_ENV_EFF', 'MAINHEAT_ENERGY_EFF','MAINHEAT_ENV_EFF', 'LIGHTING_ENERGY_EFF',
           'LIGHTING_ENV_EFF','MAINHEATC_ENERGY_EFF' ,'MAINHEATC_ENV_EFF', 'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF',
           'WALLS_ENERGY_EFF', 'WALLS_ENV_EFF']] = od.fit_transform(jd[['CURRENT_ENERGY_RATING', 'HOT_WATER_ENERGY_EFF','HOT_WATER_ENV_EFF', 'MAINHEAT_ENERGY_EFF','MAINHEAT_ENV_EFF', 'LIGHTING_ENERGY_EFF',
           'LIGHTING_ENV_EFF','MAINHEATC_ENERGY_EFF' ,'MAINHEATC_ENV_EFF', 'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF',
           'WALLS_ENERGY_EFF', 'WALLS_ENV_EFF']])
jd.head()

,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,County,PPD Category Type,...,WALLS_ENV_EFF,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,CONSTITUENCY_LABEL,Year
0,35000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,A,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,mains gas - this is for backwards compatibilit...,Coventry South,1995
1,33000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,A,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,mains gas - this is for backwards compatibilit...,Coventry South,2000
2,54650,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,A,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,mains gas - this is for backwards compatibilit...,Coventry South,2001
3,115000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,A,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,mains gas - this is for backwards compatibilit...,Coventry South,2005
4,36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,WEST MIDLANDS,A,...,2.0,4.0,4.0,3.0,3.0,5.0,5.0,mains gas - this is for backwards compatibilit...,Coventry South,1995


In [47]:
#latest copy of encoded dataset 
New = jd.copy()

In [48]:
New['CURRENT_ENERGY_RATING']

0         3.0
1         3.0
2         3.0
3         3.0
4         4.0
         ... 
115367    3.0
115368    4.0
115369    3.0
115370    3.0
115371    3.0
Name: CURRENT_ENERGY_RATING, Length: 115372, dtype: float64

In [49]:
New.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115372 entries, 0 to 115371
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Price                          115372 non-null  int64  
 1   Postcode                       115372 non-null  object 
 2   Property Type                  115372 non-null  object 
 3   Old/New                        115372 non-null  object 
 4   Duration                       115372 non-null  object 
 5   PAON                           115372 non-null  object 
 6   Street_x                       115372 non-null  object 
 7   District                       115372 non-null  object 
 8   County                         115372 non-null  object 
 9   PPD Category Type              115372 non-null  object 
 10  CURRENT_ENERGY_RATING          115372 non-null  float64
 11  CURRENT_ENERGY_EFFICIENCY      115372 non-null  int64  
 12  PROPERTY_TYPE                 

In [50]:
od1 = OrdinalEncoder()

In [51]:
#Ordinal encoding for attributes with no particular ranking
New[['MAIN_FUEL','CONSTITUENCY_LABEL','ENERGY_TARIFF','TRANSACTION_TYPE', 'CONSTITUENCY', 'BUILT_FORM', 'PROPERTY_TYPE',
    'PPD Category Type','County', 'District','Street_x','PAON','Duration', 'Old/New','Property Type','Postcode']] = od1.fit_transform(New[['MAIN_FUEL','CONSTITUENCY_LABEL','ENERGY_TARIFF','TRANSACTION_TYPE', 'CONSTITUENCY', 'BUILT_FORM', 'PROPERTY_TYPE',
    'PPD Category Type','County', 'District','Street_x','PAON','Duration', 'Old/New','Property Type','Postcode']])
New.head()

,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,County,PPD Category Type,...,WALLS_ENV_EFF,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,CONSTITUENCY_LABEL,Year
0,35000,1963.0,4.0,0.0,0.0,484.0,394.0,1.0,1.0,0.0,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,13.0,2.0,1995
1,33000,1963.0,4.0,0.0,0.0,484.0,394.0,1.0,1.0,0.0,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,13.0,2.0,2000
2,54650,1963.0,4.0,0.0,0.0,484.0,394.0,1.0,1.0,0.0,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,13.0,2.0,2001
3,115000,1963.0,4.0,0.0,0.0,484.0,394.0,1.0,1.0,0.0,...,3.0,4.0,4.0,1.0,1.0,2.0,2.0,13.0,2.0,2005
4,36000,284.0,4.0,0.0,0.0,470.0,1385.0,1.0,1.0,0.0,...,2.0,4.0,4.0,3.0,3.0,5.0,5.0,13.0,2.0,1995


In [52]:
New.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115372 entries, 0 to 115371
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Price                          115372 non-null  int64  
 1   Postcode                       115372 non-null  float64
 2   Property Type                  115372 non-null  float64
 3   Old/New                        115372 non-null  float64
 4   Duration                       115372 non-null  float64
 5   PAON                           115372 non-null  float64
 6   Street_x                       115372 non-null  float64
 7   District                       115372 non-null  float64
 8   County                         115372 non-null  float64
 9   PPD Category Type              115372 non-null  float64
 10  CURRENT_ENERGY_RATING          115372 non-null  float64
 11  CURRENT_ENERGY_EFFICIENCY      115372 non-null  int64  
 12  PROPERTY_TYPE                 

In [53]:
train_df = New[New['Year'] > 2016]

test_df = New[New['Year'] <= 2016]
test_df.shape

(94616, 43)

In [54]:
train_df.shape

(20756, 43)

In [55]:
X_train = train_df.drop(['Price'],axis=1)
y_train = train_df['Price']
X_test= test_df.drop(['Price'],axis=1)
y_test= test_df['Price']

In [56]:
model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)
y_predLR = model.predict(X_test)
MAE_LR = mean_absolute_error(y_test, y_predLR)
print('MAE for LR is:', MAE_LR)

MSE_LR = mean_squared_error(y_test, y_predLR)
print('MSE for LR is:', MSE_LR)
RMSE_LR = sqrt(MSE_LR)
print('RMSEE for LR is:', RMSE_LR)
    

coefficient of determination: 0.34464270017828535
MAE for LR is: 47255.884736869375
MSE for LR is: 5145375613.646144
RMSEE for LR is: 71731.27360953619


In [57]:
modelRM = RandomForestClassifier(n_estimators=100)
modelRM.fit(X_train, y_train)
r_sqRM = modelRM.score(X_train, y_train)
print('coefficient of determination:', r_sqRM)
y_predRM = modelRM.predict(X_test)
MAE_RM = mean_absolute_error(y_test, y_predRM)
print('MAE for RM is:', MAE_RM)

MSE_RM = mean_squared_error(y_test, y_predRM)
print('MSE for RM is:', MSE_RM)
RMSE_RM = sqrt(MSE_RM)
print('RMSEE for RM is:', RMSE_RM)
    

coefficient of determination: 0.9968201965696666
MAE for RM is: 77932.47427496407
MSE for RM is: 14121196434.178722
RMSEE for RM is: 118832.64044099467


In [58]:
modelXGB = XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1.0, colsample_bytree=1.0)
modelXGB.fit(X_train, y_train)
r_sqXGB = modelXGB.score(X_train, y_train)
print('coefficient of determination for XGB:', r_sqXGB)
y_predXGB = modelXGB.predict(X_test)
MAE_XGB = mean_absolute_error(y_test, y_predXGB)
print('MAE for XGB is:', MAE_XGB)

MSE_XGB = mean_squared_error(y_test, y_predXGB)
print('MSE for XGB is:', MSE_XGB)
RMSE_XGB = sqrt(MSE_XGB)
print('RMSEE for XGB is:', RMSE_XGB)
    

coefficient of determination for XGB: 0.9523944212180437
MAE for XGB is: 72698.9504176132
MSE for XGB is: 8934713517.615007
RMSEE for XGB is: 94523.61354505554


In [59]:
modelLASSO = Lasso(alpha=0.01, max_iter=10e5)
modelLASSO.fit(X_train, y_train)
r_sqLASSO = modelLASSO.score(X_train, y_train)
print('coefficient of determination for LASSO:', r_sqLASSO)
y_predLASSO = modelLASSO.predict(X_test)
MAE_LASSO = mean_absolute_error(y_test, y_predLASSO)
print('MAE for LASSO is:', MAE_LASSO)

MSE_LASSO = mean_squared_error(y_test, y_predLASSO)
print('MSE for LASSO is:', MSE_LASSO)
RMSE_LASSO = sqrt(MSE_LASSO)
print('RMSEE for LASSO is:', RMSE_LASSO)


coefficient of determination for LASSO: 0.34464270013500375
MAE for LASSO is: 47254.7842485436
MSE for LASSO is: 5145256886.059453
RMSEE for LASSO is: 71730.44601882421


In [28]:
model = ['Linear Regression','Random Forest', 'XGBOOST', 'LASSO']
R_SQUARED = [r_sq, r_sqRM, r_sqXGB, r_sqLASSO]
MAE = [MAE_LR,MAE_RM,MAE_XGB,MAE_LASSO]
MSE = [MSE_LR,MSE_RM,MSE_XGB,MSE_LASSO]
RMSE = [RMSE_LR,RMSE_RM,RMSE_XGB,RMSE_LASSO]

In [29]:
#Model Comparison
Result = pd.DataFrame()
Result['Model'] = model
Result['R_SQUARED'] = R_SQUARED
Result['MAE'] = MAE
Result['MSE'] = MSE
Result['RMSE'] = RMSE
Result

,Model,R_SQUARED,MAE,MSE,RMSE
0,Linear Regression,0.344643,47255.884737,5.145376e+09,71731.273610
1,Random Forest,0.996820,78570.958168,1.613711e+10,127031.924288
2,XGBOOST,0.952394,72698.950418,8.934714e+09,94523.613545
3,LASSO,0.344643,47254.784249,5.145257e+09,71730.446019


In [62]:
df_from_arr = pd.DataFrame(data=[y_test, y_predLR, y_predRM,y_predXGB,y_predLASSO])
pred_df = df_from_arr.T
pred_df.shape

(94616, 5)

In [63]:
#Dataframe for predicted result
pred_df.columns = ['y_test','y_predLR', 'y_predRM','y_predXGB','y_predLASSO']
pred_df.head()

,y_test,y_predLR,y_predRM,y_predXGB,y_predLASSO
0,35000.0,90763.793052,260000.0,189954.71875,90763.736737
1,33000.0,111291.926596,260000.0,189954.71875,111291.873808
2,54650.0,115397.553305,260000.0,189954.71875,115397.501222
3,115000.0,131820.060140,260000.0,189954.71875,131820.010879
4,36000.0,269654.732884,180000.0,264216.00000,269654.717024


In [64]:
pred_df.insert(0, 'ID', range(0, 0 + len(pred_df)))
pred_df.head()

,ID,y_test,y_predLR,y_predRM,y_predXGB,y_predLASSO
0,0,35000.0,90763.793052,260000.0,189954.71875,90763.736737
1,1,33000.0,111291.926596,260000.0,189954.71875,111291.873808
2,2,54650.0,115397.553305,260000.0,189954.71875,115397.501222
3,3,115000.0,131820.060140,260000.0,189954.71875,131820.010879
4,4,36000.0,269654.732884,180000.0,264216.00000,269654.717024


In [65]:
test_df = pd.read_csv('test_df.csv')
test_df.head()

,Unnamed: 0,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [66]:
test_df = test_df.drop(columns = ['Unnamed: 0'])
test_df.head()

,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,County,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,WEST MIDLANDS,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,WEST MIDLANDS,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [67]:
test_df.shape

(94616, 53)

In [68]:
test_df.insert(0, 'ID', range(0, 0 + len(test_df)))
test_df.head()

,ID,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Good,Good,Very Poor,Very Poor,Poor,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Good,Good,Average,Average,Very Good,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995


In [69]:
test_df.shape

(94616, 54)

In [70]:
#mergin prediction with test dataset
test_df_Ordinal = pd.merge(test_df, pred_df, left_on = ['ID','Price'], right_on = ['ID','y_test'])
test_df_Ordinal.head()


,ID,Transaction unique identifier,Price,Postcode,Property Type,Old/New,Duration,PAON,Street_x,District,...,LIGHTING_ENV_EFF,MAIN_FUEL,ADDRESS,CONSTITUENCY_LABEL,Year,y_test,y_predLR,y_predRM,y_predXGB,y_predLASSO
0,0,{5A77BC03-713A-4035-B903-5672E7238836},35000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,1995,35000.0,90763.793052,260000.0,189954.71875,90763.736737
1,1,{0B4E5EDA-15C3-410B-823C-4EC87A508052},33000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2000,33000.0,111291.926596,260000.0,189954.71875,111291.873808
2,2,{5337E544-5DCF-4130-B7C0-48C7916EAE55},54650,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2001,54650.0,115397.553305,260000.0,189954.71875,115397.501222
3,3,{BB19E2CF-3C1E-4C9E-8096-FFCDC4495F84},115000,cv35pq,T,N,F,36,calvert close,COVENTRY,...,Poor,mains gas - this is for backwards compatibilit...,"36, Calvert Close",Coventry South,2005,115000.0,131820.060140,260000.0,189954.71875,131820.010879
4,4,{CB2B2819-C4DB-4145-9F64-5674723FBAFF},36000,cv15ps,T,N,F,35,lower ford street,COVENTRY,...,Very Good,mains gas - this is for backwards compatibilit...,"35, Lower Ford Street",Coventry South,1995,36000.0,269654.732884,180000.0,264216.00000,269654.717024


In [71]:
test_df_Ordinal.shape

(94616, 59)

In [72]:
test_df_Ordinal.to_csv('test_df_Ordinal.csv')

In [1]:
######################################################################################
#Modelling Without year attributes
###################################################################

In [73]:
New1 = New.copy() 
New1.shape

(115372, 43)

In [74]:
train_df = New1[New1['Year'] > 2016]

test_df = New1[New1['Year'] <= 2016]


In [75]:
train_df = train_df.drop(columns=['Year'])
train_df.shape

(20756, 42)

In [76]:
test_df = test_df.drop(columns=['Year'])
test_df.shape

(94616, 42)

In [77]:
X_train = train_df.drop(['Price'],axis=1)
y_train = train_df['Price']
X_test= test_df.drop(['Price'],axis=1)
y_test= test_df['Price']

In [78]:
#LR modelling without year attribute
model = LinearRegression()
model.fit(X_train, y_train)
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)
y_predLR = model.predict(X_test)
MAE_LR = mean_absolute_error(y_test, y_predLR)
print('MAE for LR is:', MAE_LR)

MSE_LR = mean_squared_error(y_test, y_predLR)
print('MSE for LR is:', MSE_LR)
RMSE_LR = sqrt(MSE_LR)
print('RMSEE for LR is:', RMSE_LR)
    

coefficient of determination: 0.34322763984963345
MAE for LR is: 85687.49908440094
MSE for LR is: 11982981414.704828
RMSEE for LR is: 109466.80508128858


In [31]:
y_predLR

array([186333.9535888 , 186333.9535888 , 186333.9535888 , ...,
       197381.6575285 , 136482.99619589,  84349.45000876])

In [80]:
df_from_arr = pd.DataFrame(data=[y_test, y_predLR])
pred_df = df_from_arr.T
pred_df.shape

(94616, 2)

In [81]:
pred_df.columns = ['y_test','y_predLR1']
pred_df.head()

,y_test,y_predLR1
0,35000.0,186333.953589
1,33000.0,186333.953589
2,54650.0,186333.953589
3,115000.0,186333.953589
4,36000.0,366223.194127


In [82]:
pred_df.insert(0, 'ID', range(0, 0 + len(pred_df)))
pred_df.head()

,ID,y_test,y_predLR1
0,0,35000.0,186333.953589
1,1,33000.0,186333.953589
2,2,54650.0,186333.953589
3,3,115000.0,186333.953589
4,4,36000.0,366223.194127


In [84]:
test_df_Ordinal1 = pd.merge(test_df_Ordinal, pred_df, left_on = ['ID','y_test'], right_on = ['ID','y_test'])
test_df_Ordinal1.shape


(94616, 60)

In [85]:
test_df_Ordinal1.to_csv('test_df_Ordinal1.csv')